In [3]:
# !pip apt-get install -y fonts-nanum
# !pip fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

import seaborn as sns
# import missingno as mno




## Prepare data

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import warnings
warnings.simplefilter("ignore")

In [32]:
all2014 = pd.read_csv('ALL_2012-2020/all_2014.csv')
all2015 = pd.read_csv('ALL_2012-2020/all_2015.csv')
all2016 = pd.read_csv('ALL_2012-2020/all_2016.csv')
all2017 = pd.read_csv('ALL_2012-2020/all_2017.csv')
all2018 = pd.read_csv('ALL_2012-2020/all_2018.csv')
all2019 = pd.read_csv('ALL_2012-2020/all_2019.csv')
all2020 = pd.read_csv('ALL_2012-2020/all_2020.csv')

all_data = pd.concat([all2014,all2015,all2016,all2017,all2018,all2019,all2020], axis=0)
all_data.to_csv('all_2014-2020.csv', index=False)

In [27]:
def merge_data(year):
    #meta_all = pd.read_csv('META/meta_all.csv')
    asos = pd.read_csv('ALL_2012-2020/all_%d.csv' % (year))
    aws = pd.read_csv('ALL_2012-2020/all_%d.csv' % (year))
    globals()['{}_{}'.format('concat', year)] = pd.concat([asos,aws], axis=0)
    final_concat = pd.merge(globals()['concat_{}'.format(year)], meta_all, on='지점', how='left')
    final_concat.to_csv('weather_2012-2020/weather_{}.csv'.format(year), index=False)
    print('{} shape'.format(year), final_concat.shape)
    print(final_concat.위도.isnull().sum())

In [34]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23356143 entries, 0 to 4677476
Data columns (total 59 columns):
 #   Column         Dtype  
---  ------         -----  
 0   지역             object 
 1   측정소코드          int64  
 2   측정소명           object 
 3   SO2            float64
 4   CO             float64
 5   O3             float64
 6   NO2            float64
 7   PM10           float64
 8   PM25           float64
 9   주소             object 
 10  위도_x           float64
 11  경도_x           float64
 12  기상위도           float64
 13  기상경도           float64
 14  거리             float64
 15  일시             object 
 16  지점             float64
 17  지점명_x          object 
 18  기온(°C)         float64
 19  기온 QC플래그       float64
 20  강수량(mm)        float64
 21  강수량 QC플래그      float64
 22  풍속(m/s)        float64
 23  풍속 QC플래그       float64
 24  풍향(16방위)       float64
 25  풍향 QC플래그       float64
 26  습도(%)          float64
 27  습도 QC플래그       float64
 28  증기압(hPa)       float64
 29  이슬점온도(°C)    

# Null imputation

In [35]:
def find_null(df):
  df['지점'] = df['지점'].astype('object')
  null_feature = []
  for col in df.columns.tolist():
    null_count = df[col].isnull().sum()
    null_ratio = round(null_count/len(df), 2)*100
    if null_ratio >= 50:
      null_feature.append(col)
    print(col, ' : ', round(null_count/len(df), 2)*100 ,'%' )
  
  print('----- null_ratio over 50% : -----')
  print(null_feature)
  
  # null 비율이 50%이상인인 변수들은 제거
  data = df.drop(columns = null_feature)
  return data, null_feature




In [36]:
data, null_feature = find_null(all_data)

지역  :  0.0 %
측정소코드  :  0.0 %
측정소명  :  0.0 %
SO2  :  5.0 %
CO  :  6.0 %
O3  :  5.0 %
NO2  :  5.0 %
PM10  :  7.000000000000001 %
PM25  :  36.0 %
주소  :  0.0 %
위도_x  :  0.0 %
경도_x  :  0.0 %
기상위도  :  0.0 %
기상경도  :  0.0 %
거리  :  0.0 %
일시  :  0.0 %
지점  :  4.0 %
지점명_x  :  4.0 %
기온(°C)  :  4.0 %
기온 QC플래그  :  83.0 %
강수량(mm)  :  30.0 %
강수량 QC플래그  :  94.0 %
풍속(m/s)  :  4.0 %
풍속 QC플래그  :  84.0 %
풍향(16방위)  :  72.0 %
풍향 QC플래그  :  84.0 %
습도(%)  :  30.0 %
습도 QC플래그  :  83.0 %
증기압(hPa)  :  72.0 %
이슬점온도(°C)  :  72.0 %
현지기압(hPa)  :  56.00000000000001 %
현지기압 QC플래그  :  83.0 %
해면기압(hPa)  :  56.00000000000001 %
해면기압 QC플래그  :  83.0 %
일조(hr)  :  85.0 %
일조 QC플래그  :  79.0 %
일사(MJ/m2)  :  90.0 %
일사 QC플래그  :  76.0 %
적설(cm)  :  100.0 %
3시간신적설(cm)  :  100.0 %
전운량(10분위)  :  81.0 %
중하층운량(10분위)  :  80.0 %
운형(운형약어)  :  92.0 %
최저운고(100m )  :  90.0 %
시정(10m)  :  74.0 %
지면상태(지면상태코드)  :  99.0 %
현상번호(국내식)  :  95.0 %
지면온도(°C)  :  72.0 %
지면온도 QC플래그  :  83.0 %
5cm 지중온도(°C)  :  88.0 %
10cm 지중온도(°C)  :  88.0 %
20cm 지중온도(°C)  :  88.

# Outlier 

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23356143 entries, 0 to 4677476
Data columns (total 28 columns):
 #   Column   Dtype  
---  ------   -----  
 0   지역       object 
 1   측정소코드    int64  
 2   측정소명     object 
 3   SO2      float64
 4   CO       float64
 5   O3       float64
 6   NO2      float64
 7   PM10     float64
 8   PM25     float64
 9   주소       object 
 10  위도_x     float64
 11  경도_x     float64
 12  기상위도     float64
 13  기상경도     float64
 14  거리       float64
 15  일시       object 
 16  지점       object 
 17  지점명_x    object 
 18  기온(°C)   float64
 19  강수량(mm)  float64
 20  풍속(m/s)  float64
 21  습도(%)    float64
 22  풍향(deg)  float64
 23  지점명_y    object 
 24  지점주소     object 
 25  위도_y     float64
 26  경도_y     float64
 27  망        object 
dtypes: float64(18), int64(1), object(9)
memory usage: 5.0+ GB


In [62]:
data1 = data.drop(['망', '지점명_y', '위도_y' ,'경도_y'], axis=1)

In [63]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23356143 entries, 0 to 4677476
Data columns (total 24 columns):
 #   Column   Dtype  
---  ------   -----  
 0   지역       object 
 1   측정소코드    int64  
 2   측정소명     object 
 3   SO2      float64
 4   CO       float64
 5   O3       float64
 6   NO2      float64
 7   PM10     float64
 8   PM25     float64
 9   주소       object 
 10  위도_x     float64
 11  경도_x     float64
 12  기상위도     float64
 13  기상경도     float64
 14  거리       float64
 15  일시       object 
 16  지점       object 
 17  지점명_x    object 
 18  기온(°C)   float64
 19  강수량(mm)  float64
 20  풍속(m/s)  float64
 21  습도(%)    float64
 22  풍향(deg)  float64
 23  지점주소     object 
dtypes: float64(16), int64(1), object(7)
memory usage: 4.4+ GB


In [64]:
data1.shape

(23356143, 24)

In [66]:
data2 = data1.dropna(subset=['PM10'], how='all', axis=0)

In [68]:
data2.shape

(21815903, 24)

In [69]:
def plot_out(df):

  feature = [i for i in df.columns.to_list() if df[i].dtype == 'float64']


  plt.figure(figsize = (25,15))
  for i in range(len(feature)):
    h = (len(feature) // 4) + 1
    plt.subplot(h,4,i+1)
    sns.boxplot(x = feature[i],data = df)
  plt.show()


def find_out(df):

  out_count = {}
  out_idx = []

  #온도
  temp = df[df['기온(°C)'] < - 100].index.tolist()
  out_idx.extend(temp)
  out_count['기온(°C)'] = len(out_idx)
  df.loc[temp,'기온(°C)'] = np.NaN

  #습도
  humid = df[df['습도(%)'] <= -999.9].index.tolist()
  out_idx.extend(humid)
  out_count['습도(%)'] = len(humid)
  df.loc[humid,'습도(%)'] = np.NaN

  out_idx = list(set(out_idx))
  return out_count, out_idx



In [72]:
outlier, idx = find_out(data2) # 이상치 -> 결측치

In [73]:
outlier

{'기온(°C)': 4, '습도(%)': 7}

In [74]:
idx

[1241837, 1243277, 840942, 1018416, 843126, 1243997, 836574]

In [76]:
data2.shape

(21815903, 24)

# KNN imputation

In [79]:
data2.isnull().sum()

지역               0
측정소코드            0
측정소명             0
SO2         455910
CO          513402
O3          394479
NO2         275965
PM10             0
PM25       7119562
주소               0
위도_x             0
경도_x             0
기상위도             0
기상경도             0
거리               0
일시               0
지점          859320
지점명_x       859320
기온(°C)      885005
강수량(mm)    6449336
풍속(m/s)     925697
습도(%)      6446406
풍향(deg)    6928179
지점주소       2019515
dtype: int64

In [80]:
data3 = data2.drop(['PM25'], axis=1) #PM25

In [81]:
data3.head() # 지역, 측정소명, 주소, 지점명_x, 지점주소

,지역,측정소코드,측정소명,SO2,CO,O3,NO2,PM10,주소,위도_x,경도_x,기상위도,기상경도,거리,일시,지점,지점명_x,기온(°C),강수량(mm),풍속(m/s),습도(%),풍향(deg),지점주소
0,강원 강릉시,632132,옥천동,0.010,0.6,0.022,0.011,132.0,강원 강릉시 옥천동,37.758266,128.901774,37.7515,128.891,1.209623,2014-01-01 01:00,105.0,강릉,6.6,NaN,5.4,30.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
1,강원 강릉시,632132,옥천동,0.010,0.6,0.021,0.011,121.0,강원 강릉시 옥천동,37.758266,128.901774,37.7515,128.891,1.209623,2014-01-01 02:00,105.0,강릉,5.9,NaN,2.8,30.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
2,강원 강릉시,632132,옥천동,0.011,0.9,0.019,0.011,101.0,강원 강릉시 옥천동,37.758266,128.901774,37.7515,128.891,1.209623,2014-01-01 03:00,105.0,강릉,4.5,NaN,0.7,28.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
3,강원 강릉시,632132,옥천동,0.011,1.0,0.014,0.017,82.0,강원 강릉시 옥천동,37.758266,128.901774,37.7515,128.891,1.209623,2014-01-01 04:00,105.0,강릉,3.8,NaN,1.0,24.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
4,강원 강릉시,632132,옥천동,0.017,1.4,0.008,0.024,69.0,강원 강릉시 옥천동,37.758266,128.901774,37.7515,128.891,1.209623,2014-01-01 05:00,105.0,강릉,2.7,NaN,0.8,25.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소


In [83]:
data3.to_csv('data.csv', index=False)

In [87]:
data3.shape

(21815903, 23)

In [3]:
data4.shape

(21815903, 23)

In [5]:
data4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21815903 entries, 0 to 21815902
Data columns (total 23 columns):
 #   Column   Dtype  
---  ------   -----  
 0   지역       object 
 1   측정소코드    int64  
 2   측정소명     object 
 3   SO2      float64
 4   CO       float64
 5   O3       float64
 6   NO2      float64
 7   PM10     float64
 8   주소       object 
 9   위도_x     float64
 10  경도_x     float64
 11  기상위도     float64
 12  기상경도     float64
 13  거리       float64
 14  일시       object 
 15  지점       float64
 16  지점명_x    object 
 17  기온(°C)   float64
 18  강수량(mm)  float64
 19  풍속(m/s)  float64
 20  습도(%)    float64
 21  풍향(deg)  float64
 22  지점주소     object 
dtypes: float64(16), int64(1), object(6)
memory usage: 3.7+ GB


# Correlation Plot

In [ ]:
col = ['PM10','SO2','CO','O3','NO2','위도_x','경도_x','기상위도','기상경도','기온(°C)','강수량(mm)','풍속(m/s)','습도(%)','풍향(deg)']
plt.figure(figsize=(10,10))
sns.heatmap(data = data2[col].corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

In [9]:
import random
point = list(set(data4['측정소코드'].to_list()))
# test_point = random.smaple(point,   )
len(point)

574

In [17]:
# test 100개 (random)
np.random.seed(42)
test_point = random.sample(point, 100)
train_point = [p for p in point if p not in test_point]

In [18]:
train_data = data4[(data4['측정소코드'].isin(train_point))].reset_index(drop=True)
test_data = data4[(data4['측정소코드'].isin(test_point))].reset_index(drop=True)

In [20]:
train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)

In [3]:
import pandas as pd
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [4]:
train_data.isnull().sum()

지역               0
측정소코드            0
측정소명             0
SO2         333976
CO          336705
O3          367846
NO2         235772
PM10             0
주소               0
위도_x             0
경도_x             0
기상위도             0
기상경도             0
거리               0
일시               0
지점          753317
지점명_x       753317
기온(°C)      774312
강수량(mm)    5358571
풍속(m/s)     802741
습도(%)      5417622
풍향(deg)    5753077
지점주소       1790045
dtype: int64

# KNN imputation (train data)

In [5]:
data_inpute = train_data[['SO2','CO','O3','NO2','기온(°C)','강수량(mm)','풍속(m/s)','습도(%)','풍향(deg)']]

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputed = imputer.fit_transform(data_inpute)
data_imputed = pd.DataFrame(imputed, columns=data_inpute.columns)
data_imputed.info()

In [ ]:
data_imputed.isnull().sum()

In [ ]:
data_imputed.to_csv('train_knn.csv', index=False)

In [ ]:
train_data.SO2 = data_imputed.SO2
train_data.CO = data_imputed.CO
train_data.O3 = data_imputed.O3
train_data.NO2 = data_imputed.NO2
train_data.기온(°C) = data_imputed.기온(°C)
train_data.강수량(mm) = data_imputed.강수량(mm)
train_data.풍속(m/s) = data_imputed.풍속(m/s)
train_data.습도(%) = data_imputed.습도(%)
train_data.풍향(deg) = data_imputed.풍향(deg)

In [ ]:
test_impute = test_data[['SO2','CO','O3','NO2','기온(°C)','강수량(mm)','풍속(m/s)','습도(%)','풍향(deg)']]
imputed = imputer.transform(test_impute)
test_imputed = pd.DataFrame(imputed, columns=test_impute.columns)
test_imputed.info()

# Forward fill

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

import seaborn as sns
# import missingno as mno

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

import seaborn as sns
# import missingno as mnotrain = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18081790 entries, 0 to 18081789
Data columns (total 23 columns):
 #   Column   Dtype  
---  ------   -----  
 0   지역       object 
 1   측정소코드    int64  
 2   측정소명     object 
 3   SO2      float64
 4   CO       float64
 5   O3       float64
 6   NO2      float64
 7   PM10     float64
 8   주소       object 
 9   위도_x     float64
 10  경도_x     float64
 11  기상위도     float64
 12  기상경도     float64
 13  거리       float64
 14  일시       object 
 15  지점       float64
 16  지점명_x    object 
 17  기온(°C)   float64
 18  강수량(mm)  float64
 19  풍속(m/s)  float64
 20  습도(%)    float64
 21  풍향(deg)  float64
 22  지점주소     object 
dtypes: float64(16), int64(1), object(6)
memory usage: 3.1+ GB


In [4]:
train.isnull().sum()

지역               0
측정소코드            0
측정소명             0
SO2         333976
CO          336705
O3          367846
NO2         235772
PM10             0
주소               0
위도_x             0
경도_x             0
기상위도             0
기상경도             0
거리               0
일시               0
지점          753317
지점명_x       753317
기온(°C)      774312
강수량(mm)    5358571
풍속(m/s)     802741
습도(%)      5417622
풍향(deg)    5753077
지점주소       1790045
dtype: int64

In [8]:
data_impute = train[['SO2','CO','O3','NO2','기온(°C)','강수량(mm)','풍속(m/s)','습도(%)','풍향(deg)']]

In [9]:
data_impute.fillna(method='ffill', inplace=True)
data_impute.isnull().sum()

SO2           0
CO            0
O3            0
NO2           0
기온(°C)        0
강수량(mm)      14
풍속(m/s)       0
습도(%)         0
풍향(deg)    2087
dtype: int64

In [10]:
data_impute.fillna(method='ffill', inplace=True)
data_impute.isnull().sum()

SO2           0
CO            0
O3            0
NO2           0
기온(°C)        0
강수량(mm)      14
풍속(m/s)       0
습도(%)         0
풍향(deg)    2087
dtype: int64

In [18]:
data_impute['강수량(mm)'][data_impute['강수량(mm)'].isnull()==True]

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
Name: 강수량(mm), dtype: float64

In [19]:
data_impute['풍향(deg)'][data_impute['풍향(deg)'].isnull()==True]

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2082   NaN
2083   NaN
2084   NaN
2085   NaN
2086   NaN
Name: 풍향(deg), Length: 2087, dtype: float64

# Linear Interpolate

관측소별로 ts 정렬된 데이터셋마다 interpolate 적용

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

import seaborn as sns
# import missingno as mnotrain = pd.read_csv('train.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [53]:
train.head()

,지역,측정소코드,측정소명,SO2,CO,O3,NO2,PM10,주소,위도_x,...,거리,일시,지점,지점명_x,기온(°C),강수량(mm),풍속(m/s),습도(%),풍향(deg),지점주소
0,강원 강릉시,632132,옥천동,0.010,0.6,0.022,0.011,132.0,강원 강릉시 옥천동,37.758266,...,1.209623,2014-01-01 01:00,105.0,강릉,6.6,NaN,5.4,30.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
1,강원 강릉시,632132,옥천동,0.010,0.6,0.021,0.011,121.0,강원 강릉시 옥천동,37.758266,...,1.209623,2014-01-01 02:00,105.0,강릉,5.9,NaN,2.8,30.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
2,강원 강릉시,632132,옥천동,0.011,0.9,0.019,0.011,101.0,강원 강릉시 옥천동,37.758266,...,1.209623,2014-01-01 03:00,105.0,강릉,4.5,NaN,0.7,28.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
3,강원 강릉시,632132,옥천동,0.011,1.0,0.014,0.017,82.0,강원 강릉시 옥천동,37.758266,...,1.209623,2014-01-01 04:00,105.0,강릉,3.8,NaN,1.0,24.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
4,강원 강릉시,632132,옥천동,0.017,1.4,0.008,0.024,69.0,강원 강릉시 옥천동,37.758266,...,1.209623,2014-01-01 05:00,105.0,강릉,2.7,NaN,0.8,25.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소


In [48]:
len(train.측정소코드.unique()) # 474개 train 관측소

474

In [49]:
train_point = train.측정소코드.unique()

In [50]:
train_point

array([632132, 632421, 632151, 632161, 132401, 632121, 632122, 132112,
       131611, 131381, 131382, 131383, 131202, 131161, 131163, 131501,
       131502, 131471, 131472, 131473, 131241, 131571, 831151, 831152,
       831153, 831155, 131120, 131121, 131123, 131124, 131125, 131126,
       131128, 131129, 131111, 131112, 131113, 131116, 131117, 131231,
       131232, 131233, 131191, 131192, 131193, 131195, 131196, 131197,
       131198, 131581, 131141, 131142, 131144, 131145, 131561, 131621,
       131591, 131601, 131531, 131411, 131413, 131414, 131222, 131223,
       131131, 131132, 131371, 131341, 131342, 131343, 131451, 131551,
       131552, 131553, 238191, 238201, 238481, 238182, 238211, 238362,
       238131, 238132, 238133, 238111, 238141, 238142, 238143, 238144,
       238145, 238146, 238151, 238241, 238161, 437171, 437122, 437151,
       437152, 437154, 437131, 437141, 437371, 437161, 437401, 437541,
       437411, 437113, 437114, 437115, 437116, 324141, 324155, 324115,
      

In [84]:
# 관측소별로 연속형 변수 보간
for point in train_point:
    # 관측소별로 시간순 정렬된 데이터셋 생성
    df = train[train.측정소코드==point].reset_index(drop=True)
    # print(len(df))
    # print(df.isnull().sum())
    df['일시'] = pd.to_datetime(df['일시'], format='%Y-%m-%d %H:%M')
    df = df.sort_values('일시', ascending=True)
    df = df.reset_index(drop=True)
    # 일시를 인덱스로 변경
    df2 = df.copy()
    df2['일시'] = pd.to_datetime(df2['일시'])
    df2.index = df2['일시']
    del df2['일시']
    # interpolate
    cols = ['SO2','CO','O3','NO2','기온(°C)','강수량(mm)','풍속(m/s)','습도(%)','풍향(deg)']
    df_interpol = df2.copy()
    for col in cols:
        df_interpol[col] = df2[col].interpolate(method='linear', limit_direction='both')
    globals()['train_{}_interpol'.format(point)] = df_interpol

# 전체 데이터 concat
train_interpol = pd.DataFrame()
for point in train_point:
    train_interpol = pd.concat([train_interpol,globals()['train_{}_interpol'.format(point)]], axis=0)
print('=====FINAL INTERPOLATED DATASET=====')
print(train_interpol.info())
print(train_interpol.isnull().sum())

=====FINAL INTERPOLATED DATASET=====
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18081790 entries, 2014-01-01 01:00:00 to 2021-01-01 00:00:00
Data columns (total 22 columns):
 #   Column   Dtype  
---  ------   -----  
 0   지역       object 
 1   측정소코드    int64  
 2   측정소명     object 
 3   SO2      float64
 4   CO       float64
 5   O3       float64
 6   NO2      float64
 7   PM10     float64
 8   주소       object 
 9   위도_x     float64
 10  경도_x     float64
 11  기상위도     float64
 12  기상경도     float64
 13  거리       float64
 14  지점       float64
 15  지점명_x    object 
 16  기온(°C)   float64
 17  강수량(mm)  float64
 18  풍속(m/s)  float64
 19  습도(%)    float64
 20  풍향(deg)  float64
 21  지점주소     object 
dtypes: float64(16), int64(1), object(5)
memory usage: 3.1+ GB
None
지역               0
측정소코드            0
측정소명             0
SO2          64871
CO           64871
O3          175668
NO2              0
PM10             0
주소               0
위도_x             0
경도_x             0
기상위도       

In [88]:
test_point = test.측정소코드.unique()

# 관측소별로 연속형 변수 보간
for point in test_point:
    # 관측소별로 시간순 정렬된 데이터셋 생성
    df = test[test.측정소코드==point].reset_index(drop=True)
    # print(len(df))
    # print(df.isnull().sum())
    df['일시'] = pd.to_datetime(df['일시'], format='%Y-%m-%d %H:%M')
    df = df.sort_values('일시', ascending=True)
    df = df.reset_index(drop=True)
    # 일시를 인덱스로 변경
    df2 = df.copy()
    df2['일시'] = pd.to_datetime(df2['일시'])
    df2.index = df2['일시']
    del df2['일시']
    # interpolate
    cols = ['SO2','CO','O3','NO2','기온(°C)','강수량(mm)','풍속(m/s)','습도(%)','풍향(deg)']
    df_interpol = df2.copy()
    for col in cols:
        df_interpol[col] = df2[col].interpolate(method='linear', limit_direction='both')
    globals()['test_{}_interpol'.format(point)] = df_interpol

# 전체 데이터 concat
test_interpol = pd.DataFrame()
for point in test_point:
    test_interpol = pd.concat([test_interpol,globals()['test_{}_interpol'.format(point)]], axis=0)
print('=====FINAL INTERPOLATED DATASET=====')
print(test_interpol.info())
print(test_interpol.isnull().sum())

test_interpol.to_csv('test_interpol.csv', index=True)

=====FINAL INTERPOLATED DATASET=====
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3734113 entries, 2014-01-01 01:00:00 to 2021-01-01 00:00:00
Data columns (total 22 columns):
 #   Column   Dtype  
---  ------   -----  
 0   지역       object 
 1   측정소코드    int64  
 2   측정소명     object 
 3   SO2      float64
 4   CO       float64
 5   O3       float64
 6   NO2      float64
 7   PM10     float64
 8   주소       object 
 9   위도_x     float64
 10  경도_x     float64
 11  기상위도     float64
 12  기상경도     float64
 13  거리       float64
 14  지점       float64
 15  지점명_x    object 
 16  기온(°C)   float64
 17  강수량(mm)  float64
 18  풍속(m/s)  float64
 19  습도(%)    float64
 20  풍향(deg)  float64
 21  지점주소     object 
dtypes: float64(16), int64(1), object(5)
memory usage: 655.2+ MB
None
지역               0
측정소코드            0
측정소명             0
SO2          59524
CO          112876
O3               0
NO2              0
PM10             0
주소               0
위도_x             0
경도_x             0
기상위도      

In [85]:
train_interpol.isnull().sum()

지역               0
측정소코드            0
측정소명             0
SO2          64871
CO           64871
O3          175668
NO2              0
PM10             0
주소               0
위도_x             0
경도_x             0
기상위도             0
기상경도             0
거리               0
지점          753317
지점명_x       753317
기온(°C)       82247
강수량(mm)      82270
풍속(m/s)      82247
습도(%)       424378
풍향(deg)    5015578
지점주소       1790045
dtype: int64

In [86]:
train_interpol

,지역,측정소코드,측정소명,SO2,CO,O3,NO2,PM10,주소,위도_x,...,기상경도,거리,지점,지점명_x,기온(°C),강수량(mm),풍속(m/s),습도(%),풍향(deg),지점주소
일시,,,,,,,,,,,,,,,,,,,,,
2014-01-01 01:00:00,강원 강릉시,632132,옥천동,0.0100,0.6,0.022,0.0110,132.0,강원 강릉시 옥천동,37.758266,...,128.8910,1.209623,105.0,강릉,6.6,0.0,5.4,30.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
2014-01-01 02:00:00,강원 강릉시,632132,옥천동,0.0100,0.6,0.021,0.0110,121.0,강원 강릉시 옥천동,37.758266,...,128.8910,1.209623,105.0,강릉,5.9,0.0,2.8,30.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
2014-01-01 03:00:00,강원 강릉시,632132,옥천동,0.0110,0.9,0.019,0.0110,101.0,강원 강릉시 옥천동,37.758266,...,128.8910,1.209623,105.0,강릉,4.5,0.0,0.7,28.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
2014-01-01 04:00:00,강원 강릉시,632132,옥천동,0.0110,1.0,0.014,0.0170,82.0,강원 강릉시 옥천동,37.758266,...,128.8910,1.209623,105.0,강릉,3.8,0.0,1.0,24.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
2014-01-01 05:00:00,강원 강릉시,632132,옥천동,0.0170,1.4,0.008,0.0240,69.0,강원 강릉시 옥천동,37.758266,...,128.8910,1.209623,105.0,강릉,2.7,0.0,0.8,25.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 20:00:00,인천 옹진군,831494,연평도,0.0023,0.3,0.037,0.0045,19.0,인천 옹진군 연평면 연평리 631,37.666352,...,125.6982,0.902350,501.0,대연평,-5.4,0.0,3.6,69.5,23.1,인천광역시 옹진군연평면 연평리493-12
2020-12-31 21:00:00,인천 옹진군,831494,연평도,0.0022,0.3,0.036,0.0046,16.0,인천 옹진군 연평면 연평리 631,37.666352,...,125.6982,0.902350,501.0,대연평,-6.0,0.0,2.3,76.2,14.3,인천광역시 옹진군연평면 연평리493-12
2020-12-31 22:00:00,인천 옹진군,831494,연평도,0.0022,0.3,0.036,0.0045,16.0,인천 옹진군 연평면 연평리 631,37.666352,...,125.6982,0.902350,501.0,대연평,-6.1,0.0,2.3,78.9,24.1,인천광역시 옹진군연평면 연평리493-12


In [87]:
train_interpol.to_csv('train_interpol.csv', index=True)

In [80]:
len(train_interpol)

57420

In [83]:
df_interpol.isnull().sum()

지역             0
측정소코드          0
측정소명           0
SO2            0
CO             0
O3             0
NO2            0
PM10           0
주소             0
위도_x           0
경도_x           0
기상위도           0
기상경도           0
거리             0
지점            11
지점명_x         11
기온(°C)         0
강수량(mm)        0
풍속(m/s)        0
습도(%)          0
풍향(deg)    57420
지점주소          11
dtype: int64

# Delete observatories with only null

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

import seaborn as sns
# import missingno as mno

In [60]:
train_inter = pd.read_csv('train_inter')

,지역,측정소코드,측정소명,SO2,CO,O3,NO2,PM10,주소,위도_x,...,거리,일시,지점,지점명_x,기온(°C),강수량(mm),풍속(m/s),습도(%),풍향(deg),지점주소
0,강원 강릉시,632132,옥천동,0.010,0.6,0.022,0.011,132.0,강원 강릉시 옥천동,37.758266,...,1.209623,2014-01-01 01:00:00,105.0,강릉,6.6,NaN,5.4,30.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
1,강원 강릉시,632132,옥천동,0.010,0.6,0.021,0.011,121.0,강원 강릉시 옥천동,37.758266,...,1.209623,2014-01-01 02:00:00,105.0,강릉,5.9,NaN,2.8,30.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
2,강원 강릉시,632132,옥천동,0.011,0.9,0.019,0.011,101.0,강원 강릉시 옥천동,37.758266,...,1.209623,2014-01-01 03:00:00,105.0,강릉,4.5,NaN,0.7,28.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
3,강원 강릉시,632132,옥천동,0.011,1.0,0.014,0.017,82.0,강원 강릉시 옥천동,37.758266,...,1.209623,2014-01-01 04:00:00,105.0,강릉,3.8,NaN,1.0,24.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
4,강원 강릉시,632132,옥천동,0.017,1.4,0.008,0.024,69.0,강원 강릉시 옥천동,37.758266,...,1.209623,2014-01-01 05:00:00,105.0,강릉,2.7,NaN,0.8,25.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57415,강원 강릉시,632132,옥천동,0.003,0.3,0.029,0.014,24.0,강원 강릉시 경강로 2179(옥천동 327-2),37.760031,...,1.417000,2020-12-31 20:00:00,105.0,강릉,-4.3,NaN,6.4,41.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
57416,강원 강릉시,632132,옥천동,0.003,0.3,0.028,0.014,21.0,강원 강릉시 경강로 2179(옥천동 327-2),37.760031,...,1.417000,2020-12-31 21:00:00,105.0,강릉,-4.8,NaN,5.0,43.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
57417,강원 강릉시,632132,옥천동,0.003,0.3,0.028,0.014,32.0,강원 강릉시 경강로 2179(옥천동 327-2),37.760031,...,1.417000,2020-12-31 22:00:00,105.0,강릉,-4.9,NaN,5.3,42.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소
57418,강원 강릉시,632132,옥천동,0.003,0.3,0.030,0.012,22.0,강원 강릉시 경강로 2179(옥천동 327-2),37.760031,...,1.417000,2020-12-31 23:00:00,105.0,강릉,-5.0,NaN,5.0,39.0,NaN,강원도 강릉시용강동 63-20강릉자동기상관측소


In [ ]:
for point in train_point:
    globals()['train_{}'.format(point)] = train[train.측정소코드==point].reset_index(drop=True)
    globals()['train_{}'.format(point)] = pd.to_datetime(globals()['train_{}'.format(point)]['일시'], format='%Y-%m-%d %H:%M')
    globals()['train_{}'.format(point)] = globals()['train_{}'.format(point)].sort_values(by = ['일시'], ascending=True)
    globals()['train_{}'.format(point)] = globals()['train_{}'.format(point)].reset_index(drop=True)
    
    globals()['train_{}'.format(point)]

In [ ]:
df['rownum'] = np.arange(df.shape[0])
df_nona = df.dropna(subset = ['value'])
f = interp1d(df_nona['rownum'], df_nona['value'])
df['linear_fill'] = f(df['rownum'])
error = np.round(mean_squared_error(df_orig['value'], df['linear_fill']), 2)
df['linear_fill'].plot(title="Linear Fill (MSE: " + str(error) +")", ax=axes[3], label='Cubic Fill', color='brown', style=".-")

In [26]:
# plt.figure(figsize = (13,6))
# plt.plot(train['일시'],train['풍향(deg)'])

In [28]:
ts=pd.Series(train['풍향(deg)'],index=train['일시'])
ts
# ts.interpolate()

일시
2014-01-01 01:00   NaN
2014-01-01 02:00   NaN
2014-01-01 03:00   NaN
2014-01-01 04:00   NaN
2014-01-01 05:00   NaN
                    ..
2020-12-31 20:00   NaN
2020-12-31 21:00   NaN
2020-12-31 22:00   NaN
2020-12-31 23:00   NaN
2021-01-01 00:00   NaN
Name: 풍향(deg), Length: 18081790, dtype: float64

In [37]:
train['풍향(deg)'].isnull().sum()

5753077

In [31]:
len(train['풍향(deg)'])

18081790

In [30]:
len(ts)

18081790

In [36]:
ts.isnull().sum()

18081790

In [33]:
ts.interpolate(method = "time")

일시
2014-01-01 01:00   NaN
2014-01-01 02:00   NaN
2014-01-01 03:00   NaN
2014-01-01 04:00   NaN
2014-01-01 05:00   NaN
                    ..
2020-12-31 20:00   NaN
2020-12-31 21:00   NaN
2020-12-31 22:00   NaN
2020-12-31 23:00   NaN
2021-01-01 00:00   NaN
Name: 풍향(deg), Length: 18081790, dtype: float64